# Creating LDA in Python

 In the last section, we learned a lot about the key idea and concepts behind LDA topic modeling. Let's now put those ideas into practice. In this section, we will be using the Gensim library to create our topic model and the PyLDAVis library to visualize it. You can install both libraries with `pip` with the following commands:
 
```
pip install gensim
```

and

```
pip install pyldavis
```

We will also need to install NLTK, or the Natural Language Toolkit, in order to get a list of stop words. We can install the library with pip:

```
pip install nltk
```

Once you have installed `NLTK`, you will need to download the list of English stop words. You can do so with the following command:

```
nltk.download('stopwords')
```

## Importing the Required Libraries and Data

Now that we have our libraries installed correctly, we can import everything.

In [25]:
import pandas as pd
from nltk.corpus import stopwords
import string
import gensim.corpora as corpora
from gensim.models import LdaModel
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

This will import the requisite model from Gensim. For this notebook, we will be using the `LdaModel` class. This class allows us to create an LDA model. Before we can populate our model, however, we must first load and clean our data.

In [2]:
df = pd.read_csv("../data/trc.csv")
df = df[["Last", "First", "Description"]]
df

,Last,First,Description
0,AARON,Thabo Simon,An ANCYL member who was shot and severely inju...
1,ABBOTT,Montaigne,A member of the SADF who was severely injured ...
2,ABRAHAM,Nzaliseko Christopher,A COSAS supporter who was kicked and beaten wi...
3,ABRAHAMS,Achmat Fardiel,Was shot and blinded in one eye by members of ...
4,ABRAHAMS,Annalene Mildred,Was shot and injured by members of the SAP in ...
...,...,...,...
20829,XUZA,Mandla,Was severely injured when he was stoned by a f...
20830,YAKA,Mbangomuni,An IFP supporter and acting induna who was sho...
20831,YALI,Khayalethu,"Was shot by members of the SAP in Lingelihle, ..."
20832,YALO,Bikiwe,An IFP supporter whose house and possessions w...


Our goal in this section will be to model all the descriptions of violence in the TRC Volume 7 Final Report. We will, therefore, grab all documents and place them into a list.

In [26]:
docs = df.Description.tolist()
docs[0]

"An ANCYL member who was shot and severely injured by SAP members at Lephoi, Bethulie, Orange Free State (OFS) on 17 April 1991. Police opened fire on a gathering at an ANC supporter's house following a dispute between two neighbours, one of whom was linked to the ANC and the other to the SAP and a councillor."

## Cleaning Documents

Now that we have our documents, let's go ahead and load up our stop words. These will be the words that we remove from our documents.

In [27]:
stop_words = stopwords.words('english')

Next, we need a function to clean our documents. The purpose of the function below is to take a single document as an input and return a cleaned sequence of words with no punctuation or stop words.

In [29]:
def clean_doc(doc):
    no_punct = ''
    for c in doc:
        if c not in string.punctuation:
            no_punct = no_punct+c
    # with list comprehension
    # no_punct = ''.join([c for c in doc if c not in string.punctuation])
    
    words = no_punct.lower().split()
    
    final_words = []
    for word in words:
        if word not in stop_words:
            final_words.append(word)
    
    # with list comprehension
    # final_words = [word for word in words if word not in stop_words]

    return final_words

Let's take a look and see what this looks like now when we run it over our first document.

In [30]:
cleaned = clean_doc(docs[0])
print(docs[0])
print(cleaned)

An ANCYL member who was shot and severely injured by SAP members at Lephoi, Bethulie, Orange Free State (OFS) on 17 April 1991. Police opened fire on a gathering at an ANC supporter's house following a dispute between two neighbours, one of whom was linked to the ANC and the other to the SAP and a councillor.
['ancyl', 'member', 'shot', 'severely', 'injured', 'sap', 'members', 'lephoi', 'bethulie', 'orange', 'free', 'state', 'ofs', '17', 'april', '1991', 'police', 'opened', 'fire', 'gathering', 'anc', 'supporters', 'house', 'following', 'dispute', 'two', 'neighbours', 'one', 'linked', 'anc', 'sap', 'councillor']


With our function created, we can now process all our documents. In the line be below, we will convert all documents into a list called `cleaned_docs`. This will be our documents that are now represented as a sequence of words.

In [31]:
cleaned_docs = [clean_doc(doc) for doc in docs]

## Create ID-Word Index

Remember, an LDA topic model cannot look at words, rather it must look at numbers in order for the algorithm to work. This means that we need to convert all words into numbers. We can do this with a bag-of-words approach. In this approach, we create an ID-Word Index. This is essentially a dictionary where each unique word has a unique number. The dictionary is sorted alphabetically.

In [32]:
id2word = corpora.Dictionary(cleaned_docs)

With the ID-Word Index created, let's query it and see what word maps to index number `250`.

In [33]:
id2word[250]

'bmw'

As we can see, it is `BMW`, a political organization in South Africa. Now that we have our dictionary, we can convert all our documents into a sequence of numbers, rather than words. We can do this via the `doc2bow()` method. 

In [34]:
corpus = [id2word.doc2bow(cleaned_doc) for cleaned_doc in cleaned_docs]

The `corpus` object now contains all the documents but represented as a bag-of-words index, rather than as a sequence of words. This is the precise data that our LDA model will expect.

In [35]:
print(corpus[0])

[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 2), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]


In order to see what these numbers correspond to, let's take a look at our first document and map each number to the ID-Word Index.

In [36]:
for num in corpus[0]:
    num = num[0]
    print(f"{num}\t{id2word[num]}")

0	17
1	1991
2	anc
3	ancyl
4	april
5	bethulie
6	councillor
7	dispute
8	fire
9	following
10	free
11	gathering
12	house
13	injured
14	lephoi
15	linked
16	member
17	members
18	neighbours
19	ofs
20	one
21	opened
22	orange
23	police
24	sap
25	severely
26	shot
27	state
28	supporters
29	two


## Creating LDA Topic Model

With our `corpus` now prepared,, we can pass it to our LDA Model. We need have a lot of parameters that we can use here, but three things are mandatory:

- `corpus` which will be our corpus object
- `id2word` which will be our ID-Word Index
- `num_topics` which will be the number of topics we want the model to find.

Now, we can train our topic model.

In [37]:
lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=100)

Training a model may take several seconds to minutes. The more documents you have, the longer it will take. Once the model is trained, we can begin to analyze the model.

## Analyze a Document

If we want to get the topics for each document in our corpus, we can do so via `get_document_topics()`. This will take a single argument, the `corpus` object. It will return a list of topics and a score for each topic.

In [44]:
topics = lda_model.get_document_topics(corpus)
len(topics)

20834

We can analyze the topics for our first document.

In [43]:
topics[0]

[(46, 0.2453331),
 (49, 0.1601346),
 (57, 0.24741054),
 (60, 0.064691655),
 (65, 0.09262718),
 (67, 0.06555787),
 (87, 0.058863647),
 (91, 0.037500985)]

These topics were assigned to the following document:

In [16]:
print(docs[0])

An ANCYL member who was shot and severely injured by SAP members at Lephoi, Bethulie, Orange Free State (OFS) on 17 April 1991. Police opened fire on a gathering at an ANC supporter's house following a dispute between two neighbours, one of whom was linked to the ANC and the other to the SAP and a councillor.


Let's now print off some of the key words for each of the top-3 topics. For each topic, we will print off the top-10 words with the `get_topic_terms()` method which will take two arguments: the topic number and the amount of words to return.

In [47]:
for topic in topics[0][:3]:
    terms = lda_model.get_topic_terms(topic[0], 10)
    print(topic)
    for num in terms:
        num = num[0]
        print(num, id2word[num])
    print()

(46, 0.24563931)
19 ofs
27 state
10 free
22 orange
1250 wife
2 anc
4 april
279 13
754 received
17 members

(49, 0.16010018)
26 shot
8 fire
21 opened
24 sap
17 members
888 funeral
13 injured
23 police
85 dead
255 march

(57, 0.24714684)
1200 students
3044 parents
1268 william’s
17 members
2040 amabutho
2560 beat
2284 associated
224 school
24 sap
237 assaulted



These results seem acceptable.

## Analyze the Topic Model

It is often difficult to gauge the quality of a topic model by simply looking at raw output. For these reasons, it is useful to be able to study a model in its entirety. We can do this with PyLDAVis, a library that was designed to display in 2-dimensional space the topics of a topic model and the key words associated with each topic.

In [49]:
vis = pyLDAvis.gensim_models.prepare(lda_model, id_docs, id2word, mds="mmds", R=30)

In [50]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
44    -0.137228  0.331389       1        1  3.560200
32    -0.024440  0.469877       2        1  3.391703
38    -0.058394  0.320393       3        1  3.257853
75    -0.231691 -0.283164       4        1  2.487656
76    -0.021365  0.242234       5        1  2.386751
...         ...       ...     ...      ...       ...
73     0.149740  0.107291      96        1  0.292228
6      0.413677  0.241535      97        1  0.261419
56    -0.155346  0.103791      98        1  0.244720
40    -0.343639 -0.348971      99        1  0.216838
98     0.407532  0.141427     100        1  0.171992

[100 rows x 5 columns], topic_info=           Term          Freq         Total  Category  logprob  loglift
347         ifp   8027.000000   8027.000000   Default  30.0000  30.0000
28   supporters  10739.000000  10739.000000   Default  29.0000  29.0000
994         tvl   1377.000000   1377.000000   Default  28.0000  28.0000
349       natal   4464.000000   4464.000000   Default  27.0000  27.0000
2           anc  11185.000000  11185.000000   Default  26.0000  26.0000
..          ...           ...           ...       ...      ...      ...
29          two      6.005527   2344.859315  Topic100  -4.8719   0.3982
34          may      4.781922   1535.294958  Topic100  -5.0997   0.5938
17      members      4.859103   8027.289538  Topic100  -5.0837  -1.0443
305        died      3.874836   1066.417025  Topic100  -5.3101   0.7479
100      killed      4.079318   4407.214431  Topic100  -5.2586  -0.6196

[4696 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
697       1  0.059547             1
697       3  0.011579             1
697       4  0.004962             1
697       8  0.047968             1
697       9  0.011579             1
...     ...       ...           ...
2855     68  0.805436  ‘sharpeville
3006     10  0.503830   ‘terrorist’
518      93  0.614500      ‘whites’
520       5  0.024642            ’s
520       7  0.936390            ’s

[10958 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[45, 33, 39, 76, 77, 20, 19, 28, 86, 31, 6, 18, 9, 50, 59, 30, 67, 94, 90, 3, 22, 87, 54, 10, 15, 40, 60, 36, 29, 4, 100, 61, 63, 27, 70, 80, 68, 16, 49, 88, 91, 96, 66, 51, 84, 55, 97, 42, 65, 48, 53, 43, 5, 85, 47, 72, 8, 98, 52, 73, 83, 35, 89, 81, 44, 17, 79, 32, 82, 34, 71, 2, 93, 11, 62, 46, 75, 26, 95, 23, 56, 12, 38, 37, 69, 24, 13, 64, 92, 14, 25, 58, 1, 21, 78, 74, 7, 57, 41, 99])

Now, we can see all 100 topics plotted in two-dimensional space and we can study the degree to which the words in each topic are unique to that topic when compared with the corpus as as whole.

While useful as a first step to exploring data, we have more powerful ways of clustering documents today. As we will see over the remainder of the chapter, other libraries and approaches that leverage state of the art language models are perhaps better suited to finding and extracting topics from your corpus.